In [ ]:
import re
import altair as alt
import pandas as pd

#alt.data_transformers.enable('json')

nParticles = 600

def readClusters(filename):
    clusters = {}
    with open(filename) as f:
        step = 0
        for line in f:
            matches = re.finditer('\((( ?\d ?)+)\)', line)
            frame = []
            count = 0
            for match in matches:
                clusterMembers = [int(i) for i in match.group(1).split()]
                frame.append(clusterMembers)
                count += len(clusterMembers)
            for _ in range(nParticles-count):
                frame.append([-1])
            clusters[step] = frame
            step += 1;
    return clusters

def readEnergy(filename):
    energies = {}
    with open(filename) as f:
        step = 0
        for line in f: 
            time, potential_energy, _,_ = line.split()
            energies[float(time)] = float(potential_energy)
    return energies

def mean(values):
    tot = 0
    count = 0
    for value in values:
        tot += value
        count += 1
    return tot/count

dt = 0.01

In [ ]:
import importlib
def getdata(path):
    m = importlib.import_module(path+'.data')
    return {
        'completeCluster': m.completeCluster,
        'rule': m.rule
    }

In [ ]:
import os
shapes = []
for _, dirs, _ in os.walk('.'):
    shapes = {d:  getdata(d) for d in dirs if not d[0] == '.'}
    break
{k: v['rule'] for k, v in shapes.items()}

In [ ]:
#readEnergy('{}/T_{}/energy.dat'.format('a', '0.01'))

In [ ]:
#tmp = readClusters('{}/T_{}/clusters.txt'.format('a', '0.01'))
#[sum(len(c) for c in t) for i,t in tmp.items()]

In [ ]:
#dirs = ['letter_J', 'robot', 'swan']
temps = ['0.01', '0.02', '0.03', '0.04', '0.05', '0.06','0.07','0.08','0.09','0.1']

data = []

for directory, shapeData in shapes.items():
    complete = shapeData['completeCluster']
    completeSize = len(complete)
    for temp in temps :
        clusters = readClusters('{}/T_{}/clusters.txt'.format(directory, temp))     
        energy = readEnergy('{}/T_{}/energy.dat'.format(directory, temp))
        for time, frame in clusters.items():
            if(time > 4000):
                break
            if(time % 100 == 0):
                nComplete = len([cluster for cluster in frame if (cluster==complete)])
                #print("Found {} complete clusters at time {} for {}".format(nComplete, time, directory))
                data.append({
                    'dir': directory,
                    'shape': directory.split('_')[0],
                    'temp': temp,
                    'timestep': time,
                    'meanSize': (mean(len(cluster) for cluster in frame) if len(frame) > 0 else 0)/completeSize,
                    'nComplete': nComplete/50,
                    'energy': energy[time*1000]
                })
            
df = pd.DataFrame(data)


In [ ]:
df.loc[df['timestep'] == 1000]

In [ ]:
alt.Chart(df.loc[df['timestep'] == 1000]).mark_area(clip=True, line=True).encode(
    alt.X('temp:Q'),
    alt.Y('nComplete:Q',
        title='Complete clusters'
    ),
    alt.Column('shape', title=None),
    alt.Color('dir')
).properties(width=150, height=200)

In [ ]:
alt.Chart(df).mark_line(clip=True).encode(
    alt.X('timestep:Q'),
    alt.Y('energy:Q',
        scale=alt.Scale(domain=(-1.5, 1)),
        title='Potential energy'
    ),
    alt.Color('temp', scale=alt.Scale(scheme="inferno")),
    alt.Column('dir', title=None)
).properties(width=150, height=200)

In [ ]:
alt.Chart(df).mark_line().encode(
    alt.Y('meanSize', title='Average cluster size', scale=alt.Scale(type='log')),
    alt.Color('temp', scale=alt.Scale(scheme="inferno")),
    alt.Column('dir', title=None),
    x='timestep:Q'
).properties(width=150, height=200)

In [ ]:
alt.Chart(df).mark_line().encode(
    alt.Y('nComplete', title='Complete clusters', axis=alt.Axis(format='.0%')),
    alt.Color('temp', scale=alt.Scale(scheme="inferno")),
    alt.Column('dir', title=None),
    x='timestep:Q'
).properties(width=150, height=200)